In [ ]:
import pandas as pd
import requests
import time
from tqdm import tqdm
import plotly.express as px
import plotly.io as pio
from sqlalchemy import create_engine
from sqlalchemy.sql import text
import boto3
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup

: 

In [ ]:
liste_ville = []
html_content = requests.get('https://one-week-in.com/35-cities-to-visit-in-france/').text
soup = BeautifulSoup(html_content, 'html.parser')
for x in range(6,41):
    liste_ville.append(soup.select("div.entry-content a")[x].get_text())
print(liste_ville)

In [ ]:
#Creation d'un Dataframe avec les noms de villes
df=pd.DataFrame(data= {'Country':liste_ville})
df

In [ ]:
#Initialisation des listes lat et lon qui prendront les coordonnées des villes
lat =[]
lon=[]
#Loop qui prend pour les 35 pays les lat et lon
for i in tqdm(range(0,35)):
    param= {'country' : 'France','q':liste_ville[i],'format':'json'}
    r = requests.get("https://nominatim.openstreetmap.org/search?",param)
    lat.append(r.json()[0]['lat'])
    lon.append(r.json()[0]['lon'])


In [ ]:
df=pd.DataFrame(data= {'id': range(0,35) ,'city':liste_ville,'latitude':lat,'longitude':lon})
df

In [ ]:
df_meteo= pd.DataFrame()
api_key='eb0e4bd4076bf5b895e823c02eb3e0d0'

for x in tqdm(range(0,35)):

    #Iteration sur les lat et lon de chaque ville pour interroger l'API
    param= {'lat' : lat[x],'lon':lon[x],'appid' : api_key,'units':'metric'}#iteration sur les listes de cordonnées
    r = requests.get("https://api.openweathermap.org/data/2.5/onecall?",param)

    for v in range (0,7) :#Pour des coordonnées precises --> recuperation des previsions meteo des 7 jours

        extract_dict = r.json()['daily'][v]
        #Suppression de quelque keys pas utile.
        del extract_dict['feels_like']
        del extract_dict['temp']
        del extract_dict['weather']
        #Ajout du nom de la ville en question dans la loop
        extract_dict['city']=liste_ville[x]
        #Ajout du id par ville
        extract_dict['id_city']=x

        df_meteo=df_meteo.append(extract_dict,ignore_index=True)


df_meteo

In [ ]:
#OPERATION SUR COLONNES DATAFRAME
df_meteo=df_meteo[['city','id_city','rain','pop','pressure','humidity','dt']]#Selection des colonnes utiles
df_meteo['rain']=df_meteo['rain'].fillna(0)#Remplacement des data NaN en 0
#Preciser les types de valeurs par colonnes
df_meteo['id_city']=df_meteo['id_city'].astype(int)
df['latitude']=df['latitude'].astype(float)
df['longitude']=df['longitude'].astype(float)
#Creation d'une colonne date issue de la colonne 'dt'
df_meteo['dt_normal']= df_meteo['dt'].apply(lambda x : time.strftime('%d/%m/%Y %H:%M:%S',  time.gmtime(x)))
df_meteo['dt_normal']= df_meteo['dt_normal'].apply(lambda x : pd.to_datetime(x))
#Chargement des fichier en local
df.to_csv('Info ville.csv')
df_meteo.to_csv('Info meteo.csv')
df_meteo

In [ ]:
#Dataframe fusionnant des infos des 2 dataframe pour utilisation de mapbox
df_propa_precipitation_per_city = df_meteo.groupby(['id_city','city']).agg({'rain': 'sum', 'pop':'sum'})
df_propa_precipitation_per_city=df_propa_precipitation_per_city.reset_index()
df_propa_precipitation_per_city['total_prediction_rain_per_city']=df_propa_precipitation_per_city['rain']+df_propa_precipitation_per_city['pop']
df_propa_precipitation_per_city['latitude']=df['latitude']
df_propa_precipitation_per_city['longitude']=df['longitude']
df_propa_precipitation_per_city=df_propa_precipitation_per_city.sort_values('total_prediction_rain_per_city')
df_propa_precipitation_per_city.to_csv('df_propa_precipitation_per_city.csv')
df_propa_precipitation_per_city

In [ ]:
destination_name = df_propa_precipitation_per_city.iloc[0,1]
destination_name

In [ ]:
class HotelsSpider(scrapy.Spider):
    # Name of your spider
    name = "hotels"

    # Starting URL
    start_urls = ['https://www.booking.com/index.fr.html']

    # Parse function for login
    def parse(self, response):
        # FormRequest used to login


            return scrapy.FormRequest.from_response(
                response,
                formdata={'ss': destination_name},
                callback=self.after_search
            )

    # Callback used after login
    def after_search(self, response):

        hotels = response.css('.sr_item')

        for h in hotels:
            yield {
                'name': h.css('.sr-hotel__name::text').get(),
                'url': "https://www.booking.com" + h.css('.hotel_name_link').attrib["href"],
                'coords': h.css('.sr_card_address_line a').attrib["data-coords"],
                'score': h.css('.bui-review-score__badge::text').get(),
                'description': h.css('.hotel_desc::text').get()

            }



        # Select the NEXT button and store it in next_page
        try:
            next_page = response.css('a.paging-next').attrib["href"]
        except KeyError:
            logging.info('No next page. Terminating crawling process.')
        else:
            yield response.follow(next_page, callback=self.after_search)


In [ ]:
filename = "2_hotels.json"

if filename in os.listdir('/Users/alexlasnier/Downloads/Data_Collection_and_Management_Project-master/'):
    os.remove( '/Users/alexlasnier/Downloads/Data_Collection_and_Management_Project-master/'+filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Chrome/84.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        '/Users/alexlasnier/Downloads/Data_Collection_and_Management_Project-master/'+filename: {"format": "json"},
    }
})

process.crawl(HotelsSpider)

process.start()

In [ ]:
#Extract du fichier enregistré + nettoyage des colonnes
data_booking=pd.read_json('src/2_hotels.json')

data_booking['name']= data_booking['name'].apply(lambda x : x.replace('\n',''))
data_booking['description']= data_booking['description'].apply(lambda x : x.replace('\n',''))
data_booking['longitude'] = data_booking['coords'].apply(lambda x : x.split(',')[0])
data_booking['latitude']=data_booking['coords'].apply(lambda x : x.split(',')[1])
del data_booking['coords']
data_booking=data_booking.dropna()
data_booking

In [ ]:
#Connection a S3
session = boto3.Session(aws_access_key_id=open('aws_access_key_id.txt').read(),
                        aws_secret_access_key=open('aws_secret_access_key.txt').read())
s3 = session.resource("s3")
s3_client = session.client('s3')
new_buck = s3.create_bucket(Bucket="datalake-resort972")
#Key=nom a la destination  Body = Nom en local
#Envoi de fichier sur s3
my_buck = "datalake-resort972"

In [ ]:
# j'ai en local une extraction de srapy contenant des infos sur tout les hotels des 35 villes que je nettoie...
all_city = pd.DataFrame()
for ville  in liste_ville:
    hotel_ville = pd.read_json('json\hotels_'+ville+'.json')
    hotel_ville['name']= hotel_ville['name'].apply(lambda x : x.replace('\n',''))
    hotel_ville['description']= hotel_ville['description'].apply(lambda x : x.replace('\n',''))
    hotel_ville['longitude'] = hotel_ville['coords'].apply(lambda x : x.split(',')[0])
    hotel_ville['latitude']=hotel_ville['coords'].apply(lambda x : x.split(',')[1])
    del hotel_ville['coords']
    del hotel_ville['url']
    del hotel_ville['description']
    hotel_ville['city']=ville
    new_buck.put_object(Key = 'hotel'+ville)
    all_city=all_city.append(hotel_ville)
    all_city = all_city=all_city.dropna()
all_city.to_csv('hostel_data_set.csv')


In [ ]:
#Envoi des fichiers des dataframes informations d'hotel + prevision meteo
s3_client.upload_file('hostel_data_set.csv', my_buck, 'hostel_data_set.csv')
s3_client.upload_file('df_propa_precipitation_per_city.csv', my_buck, 'df_propa_precipitation_per_city.csv')

#Telecharger tout les fichers necessaires depuis s3 bucket
s3_client.download_file('datalake-resort972','hostel_data_set.csv','hostel_data_set.csv')
s3_client.download_file('datalake-resort972','df_propa_precipitation_per_city.csv','df_propa_precipitation_per_city.csv')


In [ ]:
#Connection a la BD
dbuser ='postgres'
dbpass ='Etienne972'
dbhost ='database-1.cbwhzj6hzyst.us-east-1.rds.amazonaws.com'
dbname ='postgres'
engine =create_engine( f'postgresql+psycopg2://{dbuser}:{dbpass}@{dbhost}/{dbname}',echo=True)
engine

In [ ]:
#Post sur PostgresSQL des dataframe
all_city.to_sql(
    'hostel',
    engine
)
pd.read_csv('df_propa_precipitation_per_city.csv').to_sql(
    "propa_precipitation_per_city",
    engine
)

In [ ]:
#Requete TOP 5 destination
stmt = text("select city, total_prediction_rain_per_city,longitude, latitude "
            "from propa_precipitation_per_city"
            " order by total_prediction_rain_per_city ASC"
            " limit 5;")
best_place = pd.read_sql(
    stmt,
    engine
)
best_place

In [ ]:
#Requete TOP 20 des hostels sur la TOP 5
df_top_20 = pd.DataFrame()
for x in range(0,5):
    stmt = text("select hostel.name,hostel.city,hostel.score,hostel.latitude,hostel.longitude from hostel"
                " inner join propa_precipitation_per_city on hostel.city=propa_precipitation_per_city.city "
                "where propa_precipitation_per_city.city in "
                "(select * from (select city "
                "from propa_precipitation_per_city "
                "order by total_prediction_rain_per_city ASC "
                f"offset {x}) AS offset_table LIMIT 1)"
                "order by hostel.city, hostel.score desc"
                " limit 20;")
    data_booking_from_sql = pd.read_sql(
    stmt,
    engine)
    df_top_20=df_top_20.append(data_booking_from_sql)


df_top_20["score"] = [float(str(i).replace(",", ".")) for i in df_top_20["score"]]
df_top_20['latitude']=df_top_20['latitude'].astype(float)
df_top_20['longitude']=df_top_20['longitude'].astype(float)
df_top_20=df_top_20.reset_index()
del df_top_20['index']
df_top_20

In [ ]:

pio.renderers.default = "svg"
px.set_mapbox_access_token('pk.eyJ1IjoiZXRpZW5uZTk3MiIsImEiOiJja3VwaXFrZXAxZTZuMzB0aHZ2Y2d1enNzIn0.tJypjwl5fr8Qo7uqSJjwoA')
fig = px.scatter_mapbox(best_place, lat="latitude", lon="longitude",  hover_data=['city'],   color="total_prediction_rain_per_city",color_continuous_scale=px.colors.sequential.Bluered, size_max=15, zoom=5)
fig.show()

In [ ]:
pio.renderers.default = "svg"
px.set_mapbox_access_token('pk.eyJ1IjoiZXRpZW5uZTk3MiIsImEiOiJja3VwaXFrZXAxZTZuMzB0aHZ2Y2d1enNzIn0.tJypjwl5fr8Qo7uqSJjwoA')
fig = px.scatter_mapbox(df_top_20, lat="latitude", lon="longitude",  hover_data=['name','city'],   color="score",size="score",color_continuous_scale=px.colors.sequential.solar, size_max=15, zoom=5)
fig.show()